In [31]:
import os
from PIL import Image
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error as mse
from skimage.metrics import peak_signal_noise_ratio as psnr


In [61]:
from PIL import Image
import matplotlib.pyplot as plt

def print_images(jpeg_path, jp2_path):
    image_jpeg = Image.open(jpeg_path)
    image_jp2 = Image.open(jp2_path)

    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    
    axes[0].imshow(image_jpeg)
    axes[0].set_title('Imagem Original JPEG')
    axes[0].axis('off')  

    axes[1].imshow(image_jp2)
    axes[1].set_title('Imagem Comprimida JP2')
    axes[1].axis('off') 

    plt.show()

jpeg_path = './coco_imagens_40x40/000000000108.jpeg'
jp2_path = './coco_imagens_40x40_min/000000000108_min_40.jp2'

print_images(jpeg_path, jp2_path)



FileNotFoundError: [Errno 2] No such file or directory: './coco_imagens_40x40/000000000108.jpeg'

In [41]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim
from math import log10

def calculate_metrics(original, compressed):
    mse_value = np.mean((original - compressed) ** 2)
    max_pixel = 255.0
    if mse_value == 0:
        psnr_value = float('inf')
    else:
        psnr_value = 20 * log10(max_pixel / np.sqrt(mse_value))
    try:
        ssim_value = ssim(original, compressed, multichannel=True)
    except ValueError:
        ssim_value = ssim(original, compressed, multichannel=True, win_size=3) 
    return mse_value, psnr_value, ssim_value

def calculate_compression_rate(original_size, compressed_size):
    return original_size / compressed_size if compressed_size != 0 else float('inf')

def compare_and_save_images(original_path, compressed_paths, output_folder, image_id, resolution):
    original = np.array(Image.open(original_path))
    original_size = os.path.getsize(original_path)

    for quality, compressed_path in compressed_paths.items():
        if not os.path.exists(original_path) or not os.path.exists(compressed_path):
            print("Um ou ambos os arquivos não existem:", original_path, compressed_path)
            continue
        
        compressed = np.array(Image.open(compressed_path))
        mse_val, psnr_val, ssim_val = calculate_metrics(original, compressed)
        compressed_size = os.path.getsize(compressed_path)
        compression_rate = calculate_compression_rate(original_size, compressed_size)
        
        fig, axes = plt.subplots(1, 2, figsize=(12, 6))
        axes[0].imshow(original)
        axes[0].set_title('Imagem Original')
        axes[0].axis('off')
        
        axes[1].imshow(compressed)
        axes[1].set_title(f'Imagem Comprimida ({quality})')
        axes[1].axis('off')
        
        comparison_image_path = os.path.join(output_folder, f"{image_id}_comparison_{quality}_{resolution}.png")
        plt.savefig(comparison_image_path)
        plt.close()
        
        print(f"ID da Imagem: {image_id}, Qualidade: {quality}, Resolução: {resolution}")
        print(f"Imagem de comparação salva em: {comparison_image_path}")
        print(f"MSE: {mse_val:.2f}, PSNR: {psnr_val:.2f} dB, SSIM: {ssim_val:.4f}")
        print(f"Taxa de Compressão (CR): {compression_rate:.2f}")
        print(f"Tamanho Original: {original_size} bytes, Tamanho Comprimido: {compressed_size} bytes")

image_ids = [
    "000000000108", "000000000128", "000000000180", "000000000183", "000000000229",
    "000000000552", "000000001702", "000000002051", "000000003188", "000000003853",
    "000000003907", "000000004158", "000000004165", "000000004311", "000000004505",
    "000000004623", "000000005082", "000000005127", "000000005239", "000000006435",
    "000000006570", "000000006865", "000000007399", "000000007959", "000000008441",
    "000000009242", "000000011189", "000000012214", "000000012272", "000000014696"
]

base_folder = 'C:/Users/andre/IC_AUTOMATOS_COCO'
comparison_folder = os.path.join(base_folder, 'comparison_images')
os.makedirs(comparison_folder, exist_ok=True)

for image_id in image_ids:
    original_path_40 = os.path.join(base_folder, 'coco_imagens_40x40', f"{image_id}.jpeg")
    compressed_min_40 = os.path.join(base_folder, 'coco_imagens_40x40_min', f"{image_id}_min_40.jp2")
    compressed_max_40 = os.path.join(base_folder, 'coco_imagens_40x40_max', f"{image_id}_max_40.jp2")
    original_path_120 = os.path.join(base_folder, 'coco_imagens_120x120', f"{image_id}.jpeg")
    compressed_min_120 = os.path.join(base_folder, 'coco_imagens_120x120_min', f"{image_id}_min_120.jp2")
    compressed_max_120 = os.path.join(base_folder, 'coco_imagens_120x120_max', f"{image_id}_max_120.jp2")

    compare_and_save_images(original_path_40, {'min': compressed_min_40, 'max': compressed_max_40}, comparison_folder, image_id, '40')
    compare_and_save_images(original_path_120, {'min': compressed_min_120, 'max': compressed_max_120}, comparison_folder, image_id, '120')


ID da Imagem: 000000000108, Qualidade: min, Resolução: 40
Imagem de comparação salva em: C:/Users/andre/IC_AUTOMATOS_COCO\comparison_images\000000000108_comparison_min_40.png
MSE: 86.08, PSNR: 28.78 dB, SSIM: 0.6411
Taxa de Compressão (CR): 2.85
Tamanho Original: 1152 bytes, Tamanho Comprimido: 404 bytes
ID da Imagem: 000000000108, Qualidade: max, Resolução: 40
Imagem de comparação salva em: C:/Users/andre/IC_AUTOMATOS_COCO\comparison_images\000000000108_comparison_max_40.png
MSE: 53.80, PSNR: 30.82 dB, SSIM: 0.9048
Taxa de Compressão (CR): 0.28
Tamanho Original: 1152 bytes, Tamanho Comprimido: 4104 bytes
ID da Imagem: 000000000108, Qualidade: min, Resolução: 120
Imagem de comparação salva em: C:/Users/andre/IC_AUTOMATOS_COCO\comparison_images\000000000108_comparison_min_120.png
MSE: 87.26, PSNR: 28.72 dB, SSIM: 0.6595
Taxa de Compressão (CR): 9.48
Tamanho Original: 4436 bytes, Tamanho Comprimido: 468 bytes
ID da Imagem: 000000000108, Qualidade: max, Resolução: 120
Imagem de comparação

In [54]:
#gerar um dataset
import pandas as pd
import numpy as np
from PIL import Image
import os

def calculate_metrics(original, compressed):
    mse_value = np.mean((original - compressed) ** 2)
    max_pixel = 255.0
    psnr_value = 20 * np.log10(max_pixel / np.sqrt(mse_value)) if mse_value != 0 else float('inf')
    ssim_value = ssim(original, compressed, multichannel=True, win_size=3)
    return mse_value, psnr_value, ssim_value

def calculate_compression_rate(original_size, compressed_size):
    return original_size / compressed_size if compressed_size != 0 else float('inf')

def generate_comparison_data(image_ids, base_folder, resolution, quality):
    results = []
    for image_id in image_ids:
        original_path = f"{base_folder}/coco_imagens_{resolution}x{resolution}/{image_id}.jpeg"
        if not os.path.exists(original_path):
            continue
        original = np.array(Image.open(original_path))
        original_size = os.path.getsize(original_path)
        
        compressed_path = f"{base_folder}/coco_imagens_{resolution}x{resolution}_{quality}/{image_id}_{quality}_{resolution}.jp2"
        if not os.path.exists(compressed_path):
            continue
        compressed = np.array(Image.open(compressed_path))
        compressed_size = os.path.getsize(compressed_path)
        
        mse, psnr, ssim = calculate_metrics(original, compressed)
        cr = calculate_compression_rate(original_size, compressed_size)

        results.append({
            'Image ID': image_id,
            'MSE': mse,
            'PSNR': psnr,
            'SSIM': ssim,
            'Compression Rate': cr,
            'Original Size': original_size,
            'Compressed Size': compressed_size
        })
    return results

def create_excel_dataset(comparison_data, resolution, quality):
    metrics = ['MSE', 'PSNR', 'SSIM', 'Compression Rate', 'Original Size', 'Compressed Size']
    df = pd.DataFrame(index=metrics)
    
    for data in comparison_data:
        for metric in metrics:
            df.loc[metric, data['Image ID']] = data[metric]
    
    excel_path = f'C:/Users/andre/IC_AUTOMATOS_COCO/comparison_results_{resolution}_{quality}.xlsx'
    df.to_excel(excel_path)
    
    return df, excel_path

base_folder = 'C:/Users/andre/IC_AUTOMATOS_COCO'
resolutions = ['40', '120']
qualities = ['min', 'max']
image_ids = [
    "000000000108", "000000000128", "000000000180", "000000000183", "000000000229",
    "000000000552", "000000001702", "000000002051", "000000003188", "000000003853",
    "000000003907", "000000004158", "000000004165", "000000004311", "000000004505",
    "000000004623", "000000005082", "000000005127", "000000005239", "000000006435",
    "000000006570", "000000006865", "000000007399", "000000007959", "000000008441",
    "000000009242", "000000011189", "000000012214", "000000012272", "000000014696"
]


for resolution in resolutions:
    for quality in qualities:
        comparison_data = generate_comparison_data(image_ids, base_folder, resolution, quality)
        df, excel_path = create_excel_dataset(comparison_data, resolution, quality)
        print(f"Excel file for {resolution}x{resolution} {quality} has been created at: {excel_path}")


Excel file for 40x40 min has been created at: C:/Users/andre/IC_AUTOMATOS_COCO/comparison_results_40_min.xlsx
Excel file for 40x40 max has been created at: C:/Users/andre/IC_AUTOMATOS_COCO/comparison_results_40_max.xlsx
Excel file for 120x120 min has been created at: C:/Users/andre/IC_AUTOMATOS_COCO/comparison_results_120_min.xlsx



KeyboardInterrupt

